# Benchmark Date: 2025-12-10

mbo_utilities loading performance benchmarks

In [ ]:
import mbo_utilities
import platform
import os

print(f"mbo_utilities: {mbo_utilities.__version__}")
print(f"Date: 2025-12-10")
print()

# system info
print("=" * 50)
print("SYSTEM INFO")
print("=" * 50)
print(f"OS: {platform.system()} {platform.release()}")
print(f"Python: {platform.python_version()}")
print(f"CPU: {platform.processor()}")
print(f"CPU cores: {os.cpu_count()}")

# memory info
try:
    import psutil
    mem = psutil.virtual_memory()
    print(f"RAM: {mem.total / 1024**3:.1f} GB total, {mem.available / 1024**3:.1f} GB available ({mem.percent}% used)")
    
    # disk info
    disk = psutil.disk_usage('C:/')
    print(f"Disk (C:): {disk.total / 1024**3:.0f} GB total, {disk.free / 1024**3:.0f} GB free")
except ImportError:
    print("(install psutil for memory/disk info)")

# numpy info
import numpy as np
print(f"NumPy: {np.__version__}")

print()
print("Network: Testing network I/O (SMB)")
print("=" * 50)

In [30]:
from pathlib import Path
import time
import numpy as np
from tifffile import TiffFile
from mbo_utilities.metadata import get_metadata
from mbo_utilities.arrays.tiff import MboRawArray
from mbo_utilities import imread

# test path - network drive
test_path = r"\\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green"

In [31]:
p = Path(test_path)
files = sorted(list(p.glob("*.tif")) + list(p.glob("*.tiff")))
print(f"Found {len(files)} files")

Found 379 files


## Array Initialization

In [ ]:
%%timeit -n 1 -r 3
arr = MboRawArray(files=files)

Counting frames:   0%|          | 0/379 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/379 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/379 [00:00<?, ?it/s]

1.35 s ± 39.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [33]:
arr = MboRawArray(files=files)
print(f"shape: {arr.shape}")
print(f"dtype: {arr.dtype}")
print(f"num_channels (z-planes): {arr.num_channels}")
print(f"num_rois: {arr.num_rois}")
print(f"fix_phase: {arr.fix_phase}")
print(f"use_fft: {arr.use_fft}")

Counting frames:   0%|          | 0/379 [00:00<?, ?it/s]

shape: (58825, 14, 448, 448)
dtype: int16
num_channels (z-planes): 14
num_rois: 2
fix_phase: True
use_fft: False


## Indexing Shapes

Verify what each index returns:

In [ ]:
arr.fix_phase = False

# compute expected shapes without loading data
nf, nc, h, w = arr.shape

print("Indexing patterns (expected shapes):")
print(f"  arr[0].shape     = ({nc}, {h}, {w})      <- frame 0, ALL z-planes")
print(f"  arr[0, 0].shape  = ({h}, {w})            <- frame 0, z-plane 0")
print(f"  arr[:, 0].shape  = ({nf}, {h}, {w})      <- ALL frames, z-plane 0")

Indexing patterns (expected shapes):
  arr[0].shape     = (14, 448, 448)      <- frame 0, ALL z-planes
  arr[0, 0].shape  = (448, 448)            <- frame 0, z-plane 0
  arr[:, 0].shape  = (58825, 448, 448)      <- ALL frames, z-plane 0 (DON'T RUN - loads all data!)


## Single Frame Access - NO Phase Correction

In [35]:
arr.fix_phase = False
print(f"fix_phase = {arr.fix_phase}")

fix_phase = False


In [36]:
# arr[0, 0] - single frame, single z-plane
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i, 0] (single frame, single z):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

arr[i, 0] (single frame, single z):
  Mean: 5.43 ms
  Min:  2.50 ms
  Max:  28.18 ms


In [37]:
# arr[0] - single frame, ALL z-planes
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i] (single frame, ALL z-planes):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

arr[i] (single frame, ALL z-planes):
  Mean: 29.51 ms
  Min:  18.01 ms
  Max:  67.09 ms


## Single Frame Access - WITH Phase Correction

In [38]:
arr.fix_phase = True
arr.use_fft = False  # integer method
print(f"fix_phase = {arr.fix_phase}, use_fft = {arr.use_fft}")

fix_phase = True, use_fft = False


In [39]:
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i, 0] WITH phase corr (integer method):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

arr[i, 0] WITH phase corr (integer method):
  Mean: 0.98 ms
  Min:  0.89 ms
  Max:  1.39 ms


In [40]:
arr.use_fft = True  # FFT method
print(f"fix_phase = {arr.fix_phase}, use_fft = {arr.use_fft}")

times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i, 0] WITH phase corr (FFT method):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

fix_phase = True, use_fft = True
arr[i, 0] WITH phase corr (FFT method):
  Mean: 19.64 ms
  Min:  19.17 ms
  Max:  20.67 ms


## Direct TiffFile Read (baseline)

In [41]:
tf = arr.tiff_files[0]

times = []
for i in range(20):
    t0 = time.perf_counter()
    page = tf.asarray(key=i)
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"Direct tf.asarray(key=i):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

Direct tf.asarray(key=i):
  Mean: 0.13 ms
  Min:  0.05 ms
  Max:  0.22 ms


## Reduction Methods

In [42]:
arr.fix_phase = False

# test on subset of frames
n_frames = min(50, arr.num_frames)
print(f"Testing reductions on first {n_frames} frames, z-plane 0")

Testing reductions on first 50 frames, z-plane 0


In [43]:
# mean
t0 = time.perf_counter()
result = arr[:n_frames, 0].mean(axis=0)
t1 = time.perf_counter()
print(f"arr[:n, 0].mean(axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

arr[:n, 0].mean(axis=0): 132.2 ms, shape=(448, 448)


In [44]:
# max projection
t0 = time.perf_counter()
result = arr[:n_frames, 0].max(axis=0)
t1 = time.perf_counter()
print(f"arr[:n, 0].max(axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

arr[:n, 0].max(axis=0): 25.8 ms, shape=(448, 448)


In [45]:
# std
t0 = time.perf_counter()
result = arr[:n_frames, 0].std(axis=0)
t1 = time.perf_counter()
print(f"arr[:n, 0].std(axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

arr[:n, 0].std(axis=0): 58.5 ms, shape=(448, 448)


In [46]:
# percentile
t0 = time.perf_counter()
data = arr[:n_frames, 0]
result = np.percentile(data, 99, axis=0)
t1 = time.perf_counter()
print(f"np.percentile(arr[:n, 0], 99, axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

np.percentile(arr[:n, 0], 99, axis=0): 94.4 ms, shape=(448, 448)


## Multi-frame Access

In [47]:
arr.fix_phase = False

for n in [10, 50, 100]:
    if n > arr.num_frames:
        continue
    times = []
    for _ in range(3):
        t0 = time.perf_counter()
        data = arr[:n, 0]
        t1 = time.perf_counter()
        times.append((t1 - t0) * 1000)
    print(f"arr[:{n}, 0]: {np.mean(times):.1f} ms (shape={data.shape})")

arr[:10, 0]: 3.6 ms (shape=(10, 448, 448))
arr[:50, 0]: 21.9 ms (shape=(50, 448, 448))
arr[:100, 0]: 105.0 ms (shape=(100, 448, 448))


## __getitem__ Breakdown

In [48]:
from mbo_utilities.util import listify_index, index_length
from mbo_utilities.arrays.tiff import _convert_range_to_slice

arr.fix_phase = False
key = (0, 0)

# step 1: key parsing
t0 = time.perf_counter()
for _ in range(1000):
    if not isinstance(key, tuple):
        k = (key,)
    else:
        k = key
    t_key, z_key, _, _ = tuple(_convert_range_to_slice(x) for x in k) + (slice(None),) * (4 - len(k))
t1 = time.perf_counter()
print(f"Key parsing x1000: {(t1-t0)*1000:.2f} ms")

# step 2: listify_index
t0 = time.perf_counter()
for _ in range(1000):
    frames = listify_index(t_key, arr.num_frames)
    chans = listify_index(z_key, arr.num_channels)
t1 = time.perf_counter()
print(f"listify_index x1000: {(t1-t0)*1000:.2f} ms")

# step 3: process_rois (includes I/O)
t0 = time.perf_counter()
out = arr.process_rois(frames, chans)
t1 = time.perf_counter()
print(f"process_rois x1: {(t1-t0)*1000:.2f} ms")

Key parsing x1000: 0.62 ms
listify_index x1000: 0.52 ms
process_rois x1: 0.52 ms


In [49]:
# index_length vs len(listify_index)
dim_size = arr.shape[2]  # height

t0 = time.perf_counter()
for _ in range(1000):
    x = len(listify_index(slice(None), dim_size))
t1 = time.perf_counter()
print(f"len(listify_index(slice(None), {dim_size})) x1000: {(t1-t0)*1000:.2f} ms")

t0 = time.perf_counter()
for _ in range(1000):
    x = index_length(slice(None), dim_size)
t1 = time.perf_counter()
print(f"index_length(slice(None), {dim_size}) x1000: {(t1-t0)*1000:.2f} ms")

len(listify_index(slice(None), 448)) x1000: 2.75 ms
index_length(slice(None), 448) x1000: 0.95 ms


## Summary

In [50]:
print("=" * 60)
print("BENCHMARK SUMMARY")
print("=" * 60)
print(f"Date: 2025-12-10")
print(f"mbo_utilities: {mbo_utilities.__version__}")
print(f"Data: {test_path}")
print(f"Shape: {arr.shape}")
print(f"Files: {len(files)}")
print("=" * 60)

BENCHMARK SUMMARY
Date: 2025-12-10
mbo_utilities: 2.3.2
Data: \\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green
Shape: (58825, 14, 448, 448)
Files: 379
